In [35]:
import logging
import pandas as pd
import os
import numpy as np

In [36]:
from retrieve_post_by_userid import RetrievePostByUserId
t = RetrievePostByUserId(sessionid="")

In [37]:
def get_post_data(data):
    post_list = []
    if data is not None and data["data"] is not None and data["data"]["mediaData"] is not None and data["data"]["mediaData"]["edges"] is not None:
        edges = data["data"]["mediaData"]["edges"]
        try:
            for edge in edges:
                thread_items = edge["node"]["thread_items"]
                for thread_item in thread_items:
                    post = thread_item["post"]
                    if post["caption"] is None or post["user"] is None or post["text_post_app_info"] is None:
                        continue
                    current_post = {
                        "id": post.get("id",""),
                        "post_pk": post.get("pk",""),
                        "caption_text": post.get("caption",{}).get("text",""),
                        "like_count": post.get("like_count",""),
                        "taken_at": post.get("taken_at",""),
                        "username": post.get("user",{}).get("username",""),
                        "user_pk": post.get("user",{}).get("pk",""),
                        "quote_count": post["text_post_app_info"].get("quote_count",0),
                        "repost_count": post["text_post_app_info"].get("repost_count",0),
                        "reshare_count": post["text_post_app_info"].get("reshare_count",0),
                    }
                    post_list.append(current_post)
            return post_list
        except Exception as e:
            raise RuntimeError(f"Errore durante l'elaborazione dei dati dei follower: {e}")
    return []
        

def get_all_post(userid: str):
    results = []
    try:
        response = t.retrieve_thread_by_userid(userid)
        d_out = get_post_data(response)
        results.extend(d_out)
    except Exception as e:
        print(response)
        raise RuntimeError(f"Errore generale nel recupero dei follower per l'utente {userid}: {e}")
    return results


In [38]:
df = pd.read_csv("data_parte1.csv")
df['user_pk'] = df['user_pk'].astype(str)
unique_user_pk = df['user_pk'].unique()
len(unique_user_pk)

349

In [39]:
df2 = pd.read_csv("followers_data.csv")
df2['follower_pk'] = df2['follower_pk'].astype(str)
unique_user_pk2 = df2['follower_pk'].unique()
len(unique_user_pk2)

132987

In [40]:
unique_user = np.concatenate([unique_user_pk, unique_user_pk2])
len(unique_user)

133336

In [41]:
def load_processed_users(file_path):
    if os.path.exists(file_path):
        logging.info(f"Loading processed users from {file_path}")
        df = pd.read_csv(file_path)
        df['user_pk'] = df['user_pk'].astype(str)
        return set(df['user_pk'].tolist())
    return set()

def save_to_csv(df, file_path):
    try:
        if os.path.exists(file_path):
            df.to_csv(file_path, mode='a', header=False, index=False)
        else:
            df.to_csv(file_path, index=False)
        logging.info(f"Saved {len(df)} records to {file_path}")
    except Exception as e:
        logging.error(f"Error saving data to {file_path}: {e}")
        
csv_file = "post_data.csv"
processed_users = load_processed_users(csv_file)

In [ ]:
import threading
import pandas as pd
from queue import Queue

unique_users = unique_user 

def process_user(user, queue):
    posts = get_all_post(user)
    queue.put(pd.DataFrame(posts))

def main():
    queue = Queue()
    i = len(processed_users)
    threads = []
    all_data = []
    
    for user in unique_users:
        if user not in processed_users:
            i += 1
            thread = threading.Thread(target=process_user, args=(user, queue))
            threads.append(thread)
            thread.start()
            
            # Quando raggiungiamo 10 thread, aspettiamo che finiscano
            if len(threads) >= 20:
                for t in threads:
                    t.join()
                threads = []
                
                while not queue.empty():
                    all_data.append(queue.get())
                
                if all_data:
                    final_df = pd.concat(all_data, ignore_index=True)
                    save_to_csv(final_df, csv_file)
                    all_data = []
                
                print(f"Processati {i} utenti su {len(unique_users)}")
        # Attendere eventuali thread rimanenti
        for t in threads:
            t.join()
        
        while not queue.empty():
            all_data.append(queue.get())
        
        if all_data:
            final_df = pd.concat(all_data, ignore_index=True)
            save_to_csv(final_df, csv_file)
        
    print("Processo completato!")

if __name__ == "__main__":
    main()


Processati 53 utenti su 125602
Processati 80 utenti su 125602
Processati 101 utenti su 125602
Processati 122 utenti su 125602
Processati 143 utenti su 125602
Processati 164 utenti su 125602
Processati 185 utenti su 125602
Processati 209 utenti su 125602
Processati 230 utenti su 125602
Processati 253 utenti su 125602
Processati 275 utenti su 125602
Processati 296 utenti su 125602
Processati 322 utenti su 125602
Processati 345 utenti su 125602
Processati 373 utenti su 125602
Processati 396 utenti su 125602
Processati 418 utenti su 125602
Processati 443 utenti su 125602
Processati 464 utenti su 125602
Processati 496 utenti su 125602
Processati 517 utenti su 125602
Processati 538 utenti su 125602
Processati 561 utenti su 125602
Processati 593 utenti su 125602
Processati 614 utenti su 125602
Processati 636 utenti su 125602
Processati 658 utenti su 125602
Processati 684 utenti su 125602
Processati 713 utenti su 125602
Processati 736 utenti su 125602
Processati 759 utenti su 125602
Processati

Exception in thread Thread-2272 (process_user):
Traceback (most recent call last):
  File "c:\Progetti\progetto_asnm\.venv\Lib\site-packages\requests\models.py", line 974, in json
    return complexjson.loads(self.text, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\pasqu\AppData\Local\Programs\Python\Python312\Lib\json\__init__.py", line 346, in loads
    return _default_decoder.decode(s)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\pasqu\AppData\Local\Programs\Python\Python312\Lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\pasqu\AppData\Local\Programs\Python\Python312\Lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)

During handling of the above exception, another exception occurred:

Traceb

Processati 2063 utenti su 125602
Processati 2084 utenti su 125602
Processati 2107 utenti su 125602
Processati 2129 utenti su 125602
Processati 2150 utenti su 125602
Processati 2173 utenti su 125602
Processati 2194 utenti su 125602
Processati 2215 utenti su 125602
Processati 2238 utenti su 125602
Processati 2260 utenti su 125602
Processati 2282 utenti su 125602
Processati 2309 utenti su 125602
Processati 2334 utenti su 125602
Processati 2355 utenti su 125602
Processati 2376 utenti su 125602
Processati 2397 utenti su 125602
Processati 2418 utenti su 125602
Processati 2439 utenti su 125602
Processati 2462 utenti su 125602
Processati 2492 utenti su 125602
Processati 2535 utenti su 125602
Processati 2558 utenti su 125602
Processati 2582 utenti su 125602
Processati 2606 utenti su 125602
Processati 2629 utenti su 125602
Processati 2650 utenti su 125602
Processati 2674 utenti su 125602
Processati 2695 utenti su 125602
Processati 2718 utenti su 125602
Processati 2742 utenti su 125602
Processati

C:\Users\pasqu\AppData\Local\Temp\ipykernel_6500\71755803.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(all_data, ignore_index=True)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_6500\71755803.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(all_data, ignore_index=True)


Processati 3863 utenti su 125602


...

C:\Users\pasqu\AppData\Local\Temp\ipykernel_6500\71755803.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(all_data, ignore_index=True)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_6500\71755803.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(all_data, ignore_index=True)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_6500\71755803.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future ve

Processati 4829 utenti su 125602


C:\Users\pasqu\AppData\Local\Temp\ipykernel_6500\71755803.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(all_data, ignore_index=True)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_6500\71755803.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(all_data, ignore_index=True)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_6500\71755803.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future ve

Processati 4850 utenti su 125602
Processati 4871 utenti su 125602


C:\Users\pasqu\AppData\Local\Temp\ipykernel_6500\71755803.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(all_data, ignore_index=True)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_6500\71755803.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(all_data, ignore_index=True)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_6500\71755803.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future ve

In [9]:
df1 = pd.read_csv("post_data.csv")
print(df1.shape)
df1

(128495, 10)


,id,post_pk,caption_text,like_count,taken_at,username,user_pk,quote_count,repost_count,reshare_count
0,3557970465522674878_187264532,3557970465522674878,A little over 24 hours left to vote.\n\nhttps:...,0,1738363153,mlssfshn,187264532,0.0,0.0,NaN
1,3557782854629238181_187264532,3557782854629238181,"Galaxy Quest \nLots of cutting holes, playing ...",1,1738340790,mlssfshn,187264532,0.0,0.0,NaN
2,3557776470965660229_187264532,3557776470965660229,Black Hole Sun\nOver 5 pounds of cornhusk in m...,0,1738340029,mlssfshn,187264532,0.0,0.0,NaN
3,3556000079513982405_187264532,3556000079513982405,Over due to post videos of my work for @salina...,0,1738128266,mlssfshn,187264532,0.0,0.0,NaN
4,3555273435077793954_187264532,3555273435077793954,My son has a good point.\nWhy are we calling m...,108,1738041642,mlssfshn,187264532,1.0,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...
128490,3391930657277617664_1325688550,3391930657277617664,Happy Fathers Day to all the dads on here much...,0,1718569665,0ffwhite05,1325688550,0.0,0.0,NaN
128491,3366520930858185719_1325688550,3366520930858185719,Add The Sc VansOnMyFeet05 😜🤘🏻,0,1715540590,0ffwhite05,1325688550,0.0,0.0,NaN
128492,3366490197977515715_1325688550,3366490197977515715,Happy Mothers Day to all the moms on here!❤️🙂❤️,0,1715536926,0ffwhite05,1325688550,0.0,0.0,NaN
128493,3156609124150157370_1325688550,3156609124150157370,Sheeesh,0,1690517152,0ffwhite05,1325688550,0.0,0.0,NaN
